In [1]:
from globals import *
import topologies.DDF as DDF
import cvxpy as cp
# import pickle
import lp_load_balancing.LP_cvspy as LP

In [5]:
topo_name='ddf'
config=(36,5)
_network = DDF.DDFtopo(config[0], config[1])
_result, _graph, paths_dict =calculate_data_shortest_paths(_network, config)
print(f"calculation done for {topo_name} {config}")
# pickle.dump(_result, open(f'./pickled_data/statistics/all_shortest_paths_{config}_{topo_name}_uniform_flow.pickle', 'wb'))
# pickle.dump(_graph, open(f'./pickled_data/graphs_and_paths/{config}_{topo_name}_edgelist.pickle', 'wb'))
# pickle.dump(paths_dict, open(f'./pickled_data/graphs_and_paths/all_shortest_paths_{config}_{topo_name}_paths.pickle', 'wb'))

display(_result)
# weighted_paths=LP.LP_load_balancing(all_shortest_paths, edge_list, _integer=True, _solver=cp.CVXOPT)
data, problem=LP.Generate_load_balancing_problem(paths_dict, _graph, _integer=False)
problem.solve( verbose=True, solver=cp.GLPK)

all_weighted_paths={}

if problem.status not in ["infeasible", "unbounded"]:
    # Otherwise, problem.value is inf or -inf, respectively.
    print("Optimal objective funtion found: %s" % problem.value)
    for i, variable in enumerate(problem.variables()[0]):
        source, dest=data[i][0]
        if not( (source, dest) in all_weighted_paths.keys() ):
            all_weighted_paths[(source, dest)]=[]
        
        all_weighted_paths[(source, dest)].append( (data[i][1], variable.value) )
    print("problem solved with cp.", problem.solver_stats.solver_name)
else:
    print(problem.status)

link_load_dict=_network.distribute_uniform_flow_on_weighted_paths(all_weighted_paths)
_load_dict=list(link_load_dict.values())
_load_min=min(_load_dict)
_load_max=max(_load_dict)
_load_mean=mean(_load_dict)
_average_path_lengths, _num_paths=process_weighted_path_dict(all_weighted_paths)
_average_path_length_min=min(_average_path_lengths)
_average_path_length_max=max(_average_path_lengths)
_average_path_length_mean=mean(_average_path_lengths)
_num_paths_min=min(_num_paths)
_num_paths_max=max(_num_paths)
_num_paths_mean=mean(_num_paths)
print("after LP optimization, path length distribution is (min, ave, max):", _average_path_length_min, _average_path_length_mean, _average_path_length_max)
print("after LP optimization, path diversity distribution is (min, ave, max):", _num_paths_min, _num_paths_mean, _num_paths_max)
print("after LP optimization, link load distribution is (min, ave, max):", _load_min, _load_mean, _load_max)


print("DDF unipath:")

_network = DDF.DDFtopo(config[0], config[1])
_result, _graph, paths_dict =calculate_DDF_routing(_network, config)
print(f"calculation done for {topo_name} {config}")
# pickle.dump(_result, open(f'./pickled_data/statistics/unipath_{config}_{topo_name}_uniform_flow.pickle', 'wb'))
# # pickle.dump(_graph, open(f'./pickled_data/graphs_and_paths/{config}_{topo_name}_edgelist.pickle', 'wb'))
# pickle.dump(paths_dict, open(f'./pickled_data/graphs_and_paths/unipath_{config}_{topo_name}_paths.pickle', 'wb'))

display(_result)


calculation done for (36, 5) with shortest paths routing
calculation done for ddf (36, 5)


{'diameter': 3,
 'ave_path_length_statistics': [1.0, 2.342857142857143, 3.0],
 'num_paths_statistics': [1, 1.542857142857143, 4],
 'link_load_statistics': [12.166666666666666, 16.4, 23.083333333333336]}

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Aug 30 11:08:45 AM: Your problem has 1944 variables, 3 constraints, and 0 parameters.
(CVXPY) Aug 30 11:08:45 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 30 11:08:45 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 30 11:08:45 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 30 11:08:45 AM: Compiling problem (target solver=GLPK).
(CVXPY) Aug 30 11:08:45 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

{'diameter': 3,
 'ave_path_length_statistics': [1.0, 2.3714285714285714, 3.0],
 'num_paths_statistics': [1, 1, 1],
 'link_load_statistics': [16.0, 16.6, 17.0]}